In [ ]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
import matplotlib as plt
from sklearn.model_selection import train_test_split
import time
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import confusion_matrix
import os
import random


In [ ]:
ecg_route = "C:/Users/leo/Desktop/machine learning/project/wfdb-python-master/psr-20ms"
ecg_route2 = "C:/Users/leo/Desktop/machine learning/project/wfdb-python-master/psr-20ms/"
expand_route = "C:/Users/leo/Desktop/machine learning/project/wfdb-python-master/psr-20ms-expand"
expand_route2 = "C:/Users/leo/Desktop/machine learning/project/wfdb-python-master/psr-20ms-expand/"

In [ ]:
#binary classfication
import numpy as np
import pandas as pd

train_x = np.zeros((2000,400,400,3),int)
train_y = np.zeros((2000),int)
test_x = np.zeros((300,400,400,3),int)
test_y = np.zeros((300),int)

In [ ]:
#binary classfication
from sklearn.externals._pilutil import imread
label_route="C:/Users/leo/Desktop/machine learning/project/wfdb-python-master/training2017/REFERENCE-original.csv"
data=pd.read_csv(label_route)
class_name=['N','A']
count_test=0
count_train=0
count_train_n=0
count_train_a=0
count_test_n=0
count_test_a=0
val=0
list_test=[] 
for root, dirs, files in os.walk(ecg_route):
    random.shuffle(files)
    for file in files:
        name=file[1:6]
        num = int(name)
        I = imread(ecg_route2+file)
        #n = random.random()
        #if count<1200:
        if data['type'][num-1]=='N' and count_train_n<1000:
            train_x[count_train] = I
            train_y[count_train] = class_name.index(data['type'][num-1])
            count_train_n+=1
            count_train+=1
            #count += 1
        elif data['type'][num-1]=='N' and count_train_n>=1000 and count_test_n<150:
            test_x[count_test] = I
            test_y[count_test] = class_name.index(data['type'][num-1])
            count_test_n+=1
            count_test+=1   
        elif data['type'][num-1]=='A' and count_test_a<150:
            test_x[count_test] = I
            test_y[count_test] = class_name.index(data['type'][num-1])
            count_test_a+=1
            count_test += 1
            list_test.append(num)
        elif data['type'][num-1]=='A' and count_test_a>=150 and count_train_a<1000:
            if num not in list_test:
                train_x[count_train] = I
                train_y[count_train] = class_name.index(data['type'][num-1])
                count_train_a+=1
                count_train += 1 

for root, dirs, files in os.walk(expand_route):
    random.shuffle(files)
    for file in files:
        name=file[1:6]
        num = int(name)
        I = imread(expand_route2+file)
        if data['type'][num-1]=='A' and count_train_a<1000:
            if num not in list_test:
                train_x[count_train] = I
                train_y[count_train] = class_name.index(data['type'][num-1])
                count_train_a+=1
                count_train+=1 
        elif data['type'][num-1]=='A' and count_train_a>=1000 and count_test_a<150:
            test_x[count_test] = I
            test_y[count_test] = class_name.index(data['type'][num-1])
            count_test_a+=1
            count_test += 1
                
classes = np.unique(train_y)
nClasses = len(classes)

In [ ]:
import matplotlib.pyplot as plt
def show_results(nn_model_train): # plot performance over the training epochs
    accuracy     = nn_model_train.history['accuracy']
    val_accuracy = nn_model_train.history['val_accuracy']
    loss         = nn_model_train.history['loss']
    val_loss     = nn_model_train.history['val_loss']
    epochs       = range(len(accuracy))
    nb_epochs    = len(epochs)

    f2 = plt.figure(2)
    plt.subplot(1,2,1)
    plt.axis((0,nb_epochs,0,1.2))
    plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1,2,2)
    plt.axis((0,nb_epochs,0,1.2))
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.draw()
    plt.pause(0.001)

In [ ]:
def predict_and_visualize_results(model, input_X, output_Y):
    predicted_classes = model.predict(input_X) # Computes for every image in the test dataset
                                              # a probability distribution over the 10 categories
    predicted_classes = np.argmax(np.round(predicted_classes),axis=1) # Choose the prediction with the highest probability
    correctIndex = np.where(predicted_classes==output_Y)[0]
    incorrectIndex = np.where(predicted_classes!=output_Y)[0]
    print("Found %d correct labels" % len(correctIndex))
    print("Found %d incorrect labels" % len(incorrectIndex))
    cm = confusion_matrix(output_Y,predicted_classes)
    print(cm)

In [ ]:
imgPixelDim = 400 # image pixel dimension

## -- Reshape the images to match the NN input format  -- ##
train_x = train_x.reshape(-1, imgPixelDim, imgPixelDim, 3) # nb of images: -1 for automatically assigned; pixels: imgPixelDim x imgPixelDim ; nb of channels: 1 for grey scale, 3 for RGB
test_x = test_x.reshape( -1, imgPixelDim, imgPixelDim, 3)

## -- Convert data type from int8 to float32 -- ##
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')

## -- Normalize the dtata: rescale the pixel values in range 0 - 1 inclusive for training purposes -- ##
train_x = train_x / 255.
test_x = test_x / 255.

## -- Change the labels from categorical to one-hot encoding -- ##
# Example: image label 7 becomes [0 0 0 0 0 0 0 1 0] ; The output neurons of the NN will be trained to match the one_hot encoded array
train_y_one_hot = to_categorical(train_y)
test_y_one_hot  = to_categorical(test_y)

# Display the change for category label using one-hot encoding
print('Original label             :', train_y[0])
print('After conversion to one-hot encoded:', train_y_one_hot[0])

## -- Split the trainAndVal data into training dataset and validation dataset -- ##
# The moodel is trained over the training dataset
# The validation dataset is used to monitor when the model starts overfitting on the training dataset
train_X,valid_X,train_label,valid_label = train_test_split(train_x, train_y_one_hot, test_size=0.2, random_state=13)

In [ ]:
#---------------------------------------------------------------------------------
#------------------------------------ PROBLEM ------------------------------------
#---------------------------------------------------------------------------------
if True:
    #------------------------------Hyper Parameters-----------------------------------
    batch_size  = 6 # how many images with their corresponding cotegories to use per
    # per NN weights update step
    epochs      = 5  # how many times to loop over the entire training dataset
    # example: for a batch_size=64 and training dataset size of 48000
    # then each epoch will consist of 48000/64=750 updates of the network weights
    learning_rate = 0.00096

    model = Sequential()
    # Shapes (None, 3) and (None, 200, 200, 2) are incompatible
    #------------------------------------Architecture---------------------------------
    model.add(Conv2D(10, kernel_size=(3, 3),activation='relu',input_shape=(400, 400, 3),padding='same'))
    model.add(Conv2D(10, kernel_size=(3, 3), activation='relu', input_shape=(400, 400, 3), padding='same'))
    model.add(MaxPooling2D((2, 2),padding='same'))

    model.add(Conv2D(40, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(40, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), padding='same'))

    model.add(Dropout(0.5))

    model.add(Flatten())
    # ---------------------------------------------------------------------------------

    model.add(Dense(nClasses, activation='softmax'))

    model.summary()

    #------------------------------Optimizer-----------------------------------
    opt = keras.optimizers.RMSprop(lr=learning_rate)
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])  # Classification accuracy is the number of correct predictions made divided by the
                                

In [ ]:
#---------------------------------------------------------------------------------
#---------------------------------- PROBLEM ---- ---------------------------------
#---------------------------------------------------------------------------------

############################################################
## -- Test the performmance of the untrained model over the test dataset -- ##
predicted_classes = model.predict(test_x) # Computes for every image in the test dataset
                                                  # a probability distribution over the 10 categories
predicted_classes = np.argmax(np.round(predicted_classes),axis=1) # Choose the prediction with the highest probability

correctIndex = np.where(predicted_classes==test_y)[0]
incorrectIndex = np.where(predicted_classes!=test_y)[0]
print("Found %d correct labels using the untrained model" % len(correctIndex))
print("Found %d incorrect labels using the untrained model" % len(incorrectIndex))

## -- Train the Neural Network -- ##
start_time = time.time()
fashion_train_dropout = model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
trainTime = (time.time() - start_time)
print('Training time = {}'.format(trainTime))

## -- Test the performmance of the trained model over the test dataset -- ##
test_eval = model.evaluate(test_x, test_y_one_hot, verbose=0)
print('Test loss:', test_eval[0])      # this is the categorical_crossentropy
print('Test accuracy:', test_eval[1])  # the accuracy evaluated by the model during training and testing

show_results(fashion_train_dropout)

predict_and_visualize_results(model, test_x, test_y)
